Importing all the libraries

In [10]:
import random
import json
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

We are now going to lemmatize individual word. 

In [2]:
"""import nltk
nltk.download('punkt') 
nltk.download('wordnet') (Forgot to download it earlier)"""

"import nltk\nnltk.download('punkt') \nnltk.download('wordnet') (Forgot to download it earlier)"

In [11]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())
words=[]
classes=[]
documents=[]
ignore_letters=['?', '!','.',',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)


[(['hello'], 'greetings'), (['hey'], 'greetings'), (['hi'], 'greetings'), (['good', 'day'], 'greetings'), (['greetings'], 'greetings'), (['what', "'s", 'up'], 'greetings'), (['how', 'is', 'it', 'going'], 'greetings'), (['bye'], 'goodbye'), (['goodbye'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['take', 'care'], 'goodbye'), (['how', 'old', 'are', 'you', '?'], 'age'), (['what', "'s", 'your', 'age', '?'], 'age'), (['can', 'I', 'know', 'your', 'age', '?'], 'age'), (['what', "'s", 'your', 'name', '?'], 'name'), (['who', 'are', 'you', '?'], 'name'), (['can', 'I', 'know', 'your', 'name', '?'], 'name'), (['I', "'d", 'like', 'to', 'buy', 'something'], 'shop'), (['can', 'I', 'shop', 'here', '?'], 'shop'), (['what', 'do', 'you', 'sell', '?'], 'shop'), (['what', 'are', 'your', 'produts', '?'], 'shop'), (['what', 'are', 'your', 'working', 'hours', '?'], 'hours'), (['when', 'are', 'you', 'open', '?'], 'hours'), (['what', 'time', 'do', 'you', 'close', '?'], 'hours'), (['where', 'are', 'you'

In [12]:

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

print(words)

["'d", "'s", 'I', 'a', 'address', 'age', 'an', 'any', 'are', 'benefit', 'best', 'book', 'buy', 'bye', 'can', 'care', 'close', 'club', 'day', 'do', 'fiction', 'find', 'genre', 'going', 'good', 'goodbye', 'greeting', 'have', 'hello', 'here', 'hey', 'hi', 'hour', 'how', 'is', 'it', 'item', 'join', 'know', 'later', 'like', 'located', 'membership', 'name', 'non-fiction', 'of', 'offer', 'old', 'open', 'policy', 'popular', 'produts', 'program', 'recommend', 'return', 'see', 'sell', 'seller', 'shop', 'something', 'store', 'take', 'the', 'time', 'to', 'up', 'what', 'when', 'where', 'who', 'working', 'you', 'your']


Now we pickle and save our result.

In [13]:
classes = sorted(set(classes))
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

Next step is to the preprocessing of data and changing it to digits

In [14]:
training = []
output_empty = [0]*len(classes)

for document in documents:
    bag=[]
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    output_row= list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag, output_row])

Now we make neural network using Sequential and use dropout to avoid overfitting.

In [16]:
random.shuffle(training)
training = np.array(training, dtype=object)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size =5, verbose=1)

model.save('chatbot_model.h5', hist)
print("Done")

D:\LLM\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
D:\LLM\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Epoch 1/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.1064 - loss: 2.4079   
Epoch 2/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0300 - loss: 2.4161     
Epoch 3/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.1866 - loss: 2.3257 
Epoch 4/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1835 - loss: 2.2705    
Epoch 5/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.1563 - loss: 2.3242
Epoch 6/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3357 - loss: 2.1600 
Epoch 7/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2283 - loss: 2.2335 
Epoch 8/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2494 - loss: 2.1300     
Epoch 9/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4691 - loss: 1.9317 
Epoch 10/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.2626 - loss: 2.0845 
Epoch 11/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5723 - loss: 1.7601 
Epoch 12/200
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accurac

Done
